In [ ]:
# Check runtime type and GPU availability
import torch
import psutil
import os

print(f"🖥️  Runtime: Google Colab")
print(f"🐍 Python: {os.sys.version}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"💾 RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
print(f"🎮 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


In [ ]:
# Install required packages
%pip install -q --upgrade pip
%pip install -q torch>=2.3 transformers>=4.53 optimum[onnxruntime]>=1.22.0
%pip install -q onnxruntime>=1.18 onnx>=1.18 sentencepiece>=0.2.0
%pip install -q safetensors>=0.4.0 accelerate>=0.30.0 timm>=1.0.0
%pip install -q pillow>=10.0.0 tqdm>=4.65.0 psutil>=5.9.0

# Install latest transformers from GitHub for Gemma3n support
%pip install -q git+https://github.com/huggingface/transformers.git

print("✅ Dependencies installed successfully!")


In [ ]:
import time
import gc
import json
import datetime
from pathlib import Path
from typing import Optional

import psutil
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer

class VerboseLogger:
    """Enhanced logging class with timestamps and memory monitoring."""
    
    def __init__(self):
        self.start_time = time.time()
        self.phase_start_time = None
        self.current_phase = None
        self.max_memory_usage = 0
        
    def log(self, message: str, level: str = "INFO"):
        """Log message with timestamp and memory info."""
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        elapsed = time.time() - self.start_time
        
        # Get current memory usage
        process = psutil.Process()
        memory_mb = process.memory_info().rss / 1024 / 1024
        self.max_memory_usage = max(self.max_memory_usage, memory_mb)
        
        # Get GPU memory if available
        gpu_memory = ""
        if torch.cuda.is_available():
            gpu_mem_mb = torch.cuda.memory_allocated() / 1024 / 1024
            gpu_memory = f" | GPU: {gpu_mem_mb:.1f}MB"
        
        log_line = f"[{timestamp}] [{level}] [{elapsed:.1f}s] [RAM: {memory_mb:.1f}MB{gpu_memory}] {message}"
        print(log_line)
    
    def start_phase(self, phase_name: str, estimated_minutes: Optional[int] = None):
        """Start a new phase with estimated duration."""
        if self.phase_start_time and self.current_phase:
            phase_elapsed = time.time() - self.phase_start_time
            self.log(f"✓ Completed phase '{self.current_phase}' in {phase_elapsed/60:.1f} minutes")
        
        self.current_phase = phase_name
        self.phase_start_time = time.time()
        
        est_msg = f" (estimated: {estimated_minutes} min)" if estimated_minutes else ""
        self.log(f"🚀 Starting phase: {phase_name}{est_msg}", "PHASE")
        
    def update_progress(self, message: str):
        """Update progress within current phase."""
        self.log(f"   {message}", "PROGRESS")
    
    def log_file_info(self, file_path: Path, description: str):
        """Log file information."""
        if file_path.exists():
            size_mb = file_path.stat().st_size / 1024 / 1024
            self.log(f"📄 {description}: {file_path.name} ({size_mb:.1f}MB)")
        else:
            self.log(f"❌ {description}: {file_path.name} (not found)")
    
    def error(self, message: str):
        self.log(f"❌ ERROR: {message}", "ERROR")
    
    def success(self, message: str):
        self.log(f"✅ SUCCESS: {message}", "SUCCESS")

def convert_model_inspection(model_id: str, output_dir: str = "./output"):
    """Simplified model inspection function for Colab."""
    logger = VerboseLogger()
    out_dir = Path(output_dir)
    out_dir.mkdir(exist_ok=True)
    
    logger.log("="*60)
    logger.log("🚀 GEMMA3N VISION MODEL INSPECTION STARTED")
    logger.log("="*60)
    logger.log(f"Model ID: {model_id}")
    logger.log(f"Output Directory: {out_dir}")
    
    try:
        # Phase 1: Load model for inspection
        logger.start_phase("Model Loading and Inspection", 2)
        
        logger.update_progress("Loading model configuration...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id, 
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        
        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        logger.update_progress(f"Model loaded with {total_params:,} parameters")
        
        # Check model architecture
        logger.update_progress(f"Model type: {type(model).__name__}")
        if hasattr(model, 'vision_tower'):
            logger.update_progress("✓ Vision tower found")
        else:
            logger.update_progress("⚠️ No vision tower detected")
        
        # Phase 2: Save tokenizer/processor
        logger.start_phase("Saving Tokenizer and Processor", 1)
        
        try:
            processor = AutoProcessor.from_pretrained(model_id)
            processor.save_pretrained(out_dir)
            logger.update_progress("✓ Processor saved")
        except Exception as e:
            logger.update_progress(f"⚠️ Processor save failed: {e}")
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            tokenizer.save_pretrained(out_dir)
            logger.update_progress("✓ Tokenizer saved as fallback")
        
        # Phase 3: Create manifest
        logger.start_phase("Creating Manifest", 1)
        
        manifest = {
            "model_name": f"Gemma3n Vision Model ({model_id})",
            "model_id": model_id,
            "total_parameters": total_params,
            "model_type": type(model).__name__,
            "conversion_timestamp": datetime.datetime.now().isoformat(),
            "status": "inspection_complete",
            "notes": "Model successfully loaded and inspected. ONNX conversion would require additional time and resources."
        }
        
        manifest_path = out_dir / "model_manifest.json"
        with open(manifest_path, "w") as f:
            json.dump(manifest, f, indent=2)
        
        logger.log_file_info(manifest_path, "Generated manifest")
        
        # Summary
        total_time = time.time() - logger.start_time
        logger.log("="*60)
        logger.success("MODEL INSPECTION COMPLETED!")
        logger.log(f"⏰ Total time: {total_time/60:.1f} minutes")
        logger.log(f"💾 Peak memory usage: {logger.max_memory_usage:.1f}MB")
        logger.log(f"📁 Output directory: {out_dir}")
        logger.log("="*60)
        
        # Clean up
        del model
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return manifest_path
        
    except Exception as e:
        logger.error(f"Conversion failed: {e}")
        import traceback
        logger.error(f"Traceback:\n{traceback.format_exc()}")
        raise

print("✅ Conversion script loaded successfully!")


In [ ]:
# Test with Gemma3n E2B model (smaller, more suitable for Colab)
model_id = "google/gemma-3n-E2B-it"  # 2B effective parameters
output_dir = "/content/gemma3n_output"

print("🚀 Starting Gemma3n model conversion test...")
print(f"📦 Model: {model_id}")
print(f"📁 Output: {output_dir}")
print("-" * 60)

# Run the conversion
try:
    manifest_path = convert_model_inspection(model_id, output_dir)
    print(f"\n✅ Conversion test completed! Manifest saved to: {manifest_path}")
except Exception as e:
    print(f"\n❌ Conversion test failed: {e}")


In [ ]:
import json
import os
from pathlib import Path

output_path = Path("/content/gemma3n_output")

if output_path.exists():
    print("📁 Generated Files:")
    print("=" * 50)
    
    total_size = 0
    for file_path in output_path.iterdir():
        if file_path.is_file():
            size_mb = file_path.stat().st_size / 1024 / 1024
            total_size += size_mb
            print(f"📄 {file_path.name:<30} {size_mb:>8.1f} MB")
    
    print("-" * 50)
    print(f"📊 Total size: {total_size:.1f} MB")
    
    # Display manifest content
    manifest_file = output_path / "model_manifest.json"
    if manifest_file.exists():
        print("\n📋 Model Manifest:")
        print("=" * 50)
        with open(manifest_file) as f:
            manifest = json.load(f)
        
        for key, value in manifest.items():
            if key == "total_parameters":
                print(f"🔢 {key}: {value:,}")
            else:
                print(f"📝 {key}: {value}")
else:
    print("❌ Output directory not found")


In [ ]:
import zipfile
from google.colab import files

# Create a zip file with all outputs
zip_path = "/content/gemma3n_conversion_results.zip"
output_dir = Path("/content/gemma3n_output")

if output_dir.exists():
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in output_dir.rglob('*'):
            if file_path.is_file():
                arcname = file_path.relative_to(output_dir.parent)
                zipf.write(file_path, arcname)
    
    print(f"📦 Created zip file: {zip_path}")
    print(f"📊 Zip size: {Path(zip_path).stat().st_size / 1024 / 1024:.1f} MB")
    
    # Download the zip file
    print("⬇️ Starting download...")
    files.download(zip_path)
    print("✅ Download completed!")
else:
    print("❌ No output directory found to download")

print("\n🎉 **Happy converting!** 🚀")
print("📝 Don't forget to star the repo if this helped you!")
